In [6]:
# Import library yang diperlukan
import os
import pandas as pd

# Bikin variable pathnya biar gampang dipanggil
source_folder_path = r"C:\Users\N I T R O   5\Documents\Bootcamps\Purwadhika\Capstone\data_sources\data_sources\data_products"
output_folder_path = r"D:\PURWA\1"



In [7]:
# Fungsi buat extract data dari CSVnya dan di upload ke dataframe
def extract_data(file_path, delete_index=True):
    df = pd.read_csv(file_path)
    if delete_index:
        df.reset_index(drop=True, inplace=True)  # Remove index if required
    return df



In [8]:
# Fungsi buat transformasi data
def transform_data(df):
    # Delete simbol currency nya jadi cuma angka aja buat kolom actual price
    if 'actual_price' in df.columns:
        df['actual_price'] = df['actual_price'].replace('[^\d.]', '', regex=True)
        df['actual_price'] = pd.to_numeric(df['actual_price'], errors='coerce')  # Convert to numeric, kalau ada error dijadiin NaN
        df['actual_price'].fillna(0, inplace=True)  # ganti NaN dengan 0 
    
    # Delete simbol currency nya jadi cuma angka aja buat kolom discount price
    if 'discount_price' in df.columns:
        df['discount_price'] = df['discount_price'].replace('[^\d.]', '', regex=True)
        df['discount_price'] = pd.to_numeric(df['discount_price'], errors='coerce')  # Convert to float
        df['discount_price'].fillna(0, inplace=True)  # Replace NaN values with 0
    
    # Convert rating ke numeric
    if 'ratings' in df.columns:
        df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')  # Convert to numeric, kalau ada error dijadiin NaN
    
    # Transform the 'no_of_ratings' column
    if 'no_of_ratings' in df.columns:
    # Delete semua non numeric values, dan convert values nya ke numeric
        df['no_of_ratings'] = df['no_of_ratings'].replace('[^\d]', '', regex=True)
        df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce').fillna(0)

    
    return df



In [9]:
# Fungsi untuk coba coba statistik simple
def basic_statistics(df):
    print("Basic Statistics:")
    print(df.describe())
    print("\n")

# Fungsi yang menganalisa values yang kosong
def missing_values_analysis(df):
    print("Missing Values Analysis:")
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    print(pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage}))
    print("\n")

# Fungsi menganalisa distribusi kategori
def categorical_distribution(df, column_name):
    if column_name in df.columns:
        print(f"Distribution of {column_name}:")
        print(df[column_name].value_counts())
        print("\n")

# Fungsi untuk aggregasi atau penjumlahan datanya
def numeric_aggregation(df, column_name):
    if column_name in df.columns and pd.api.types.is_numeric_dtype(df[column_name]):
        print(f"Total {column_name}: {df[column_name].sum()}")
        print(f"Average {column_name}: {df[column_name].mean()}")
        print("\n")

#Fungsi untuk ngedisplay top N values di kolomnya
def top_n_analysis(df, column_name, n=5):
    if column_name in df.columns:
        print(f"Top {n} based on {column_name}:")
        print(df.nlargest(n, column_name))
        print("\n")

# Fungsi korelasi analisis
def correlation_analysis(df):
    print("Correlation Matrix:")
    correlation = df.corr()
    print(correlation)
    print("\n")

# # Fungsi untuk deteksi outlier
# def outlier_detection(df, column_name, threshold):
#     if column_name in df.columns and pd.api.types.is_numeric_dtype(df[column_name]):
#         outliers = df[df[column_name] > threshold]
#         print(f"Number of outliers in {column_name} greater than {threshold}: {len(outliers)}")
#         print(outliers)
#         print("\n")

# Fungsi untuk cek datanya lengkap atau engga
def data_completeness(df):
    total_rows = df.shape[0]
    if total_rows == 0:
        print("The DataFrame is empty, data completeness cannot be calculated.")
    else:
        complete_rows = df.dropna().shape[0]
        completeness_percentage = (complete_rows / total_rows) * 100
        print(f"Percentage of complete rows: {completeness_percentage:.2f}%")
    print("\n")


# Fungsi top 3 product dengan rating tertinggi
def top_3_highest_rating(df):
    if 'ratings' in df.columns:
        print("Top 3 Products with the Highest Rating:")
        top_3_ratings = df.nlargest(3, 'ratings')
        print(top_3_ratings[['name', 'ratings']])
        print("\n")

# Fungsi top 3 product dengan rating terbanyak
def top_3_most_ratings(df):
    if 'no_of_ratings' in df.columns:
        print("Top 3 Products with the Most Ratings:")
        top_3_most = df.nlargest(3, 'no_of_ratings')
        print(top_3_most[['name', 'no_of_ratings']])
        print("\n")


# Funsgi untuk melihat product dengan rating terbanyak dan juga di sort dari yang tertinggi ratingannya buat top 3 aja
def top_3_most_ratings_sorted(df):
    if 'no_of_ratings' in df.columns:
        print("Top 3 Products with the Most Ratings, Sorted by Highest:")
        sorted_ratings = df.sort_values(by='no_of_ratings', ascending=False).head(3)
        print(sorted_ratings[['name', 'no_of_ratings']])
        print("\n")



In [10]:
# Fungsi ETL utama dan analisis demography 
def main():
    # Get the list of all CSV files in the source folder
    files = [file for file in os.listdir(source_folder_path) if file.endswith('.csv')]

    # Create output directory if it doesn't exist
    os.makedirs(output_folder_path, exist_ok=True)

    # Process tiap file
    for file in files:
        file_path = os.path.join(source_folder_path, file)
        
        # Step 1: Data Extraction
        df = extract_data(file_path)
        
        # Step 2: Data Transformation
        df = transform_data(df)
        
        # Step 3: Data Demography Analysis
        print(f"=== Analysis for {file} ===")
        print("=== Basic Statistics ===")
        basic_statistics(df)
        
        print("=== Missing Values Analysis ===")
        missing_values_analysis(df)
        
        # print("=== Categorical Data Distribution ===")
        # categorical_distribution(df, "category_column")  
        
        # print("=== Numeric Data Aggregation ===")
        # numeric_aggregation(df, "price_column")  
        
        # print("=== Top N Analysis ===")
        # top_n_analysis(df, "price_column", 5) 
        
        print("=== Correlation Analysis ===")
        correlation_analysis(df)
        
        # print("=== Outlier Detection ===")
        # outlier_detection(df, "price_column", 10000)  
        
        print("=== Data Completeness ===")
        data_completeness(df)
        
        print("=== Top 3 Products with the Highest Rating ===")
        top_3_highest_rating(df)
        
        print("=== Top 3 Products with the Most Ratings ===")
        top_3_most_ratings(df)
        
        print("=== Top 3 Products with the Most Ratings, Sorted ===")
        top_3_most_ratings_sorted(df)
        
        # Save the transformed data
        output_file_path = os.path.join(output_folder_path, f"Cleaned_{file}")
        df.to_csv(output_file_path, index=False)
        print(f"Transformed data saved for {file}.\n")

# Run the main function
if __name__ == "__main__":
    main()


=== Analysis for Air Conditioners.csv ===
=== Basic Statistics ===
Basic Statistics:
          ratings  no_of_ratings  discount_price   actual_price
count  423.000000     720.000000      720.000000     720.000000
mean     3.809693     144.087500    25868.743056   38066.130028
std      0.789610     564.281521    22969.750811   31589.188305
min      1.000000       0.000000        0.000000       0.000000
25%      3.500000       0.000000        0.000000       0.000000
50%      4.000000       2.000000    32999.000000   47045.000000
75%      4.200000      40.000000    41999.250000   61990.000000
max      5.000000    9577.000000   128800.000000  149000.000000


=== Missing Values Analysis ===
Missing Values Analysis:
                Missing Values  Percentage
name                         0        0.00
main_category                0        0.00
sub_category                 0        0.00
image                        0        0.00
link                         0        0.00
ratings               

C:\Users\N I T R O   5\AppData\Local\Temp\ipykernel_37400\801549112.py:39: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()


                Missing Values  Percentage
name                         0    0.000000
main_category                0    0.000000
sub_category                 0    0.000000
image                        0    0.000000
link                         0    0.000000
ratings                    538    5.618212
no_of_ratings                0    0.000000
discount_price               0    0.000000
actual_price                 0    0.000000


=== Correlation Analysis ===
Correlation Matrix:
                 ratings  no_of_ratings  discount_price  actual_price
ratings         1.000000       0.060954        0.114363      0.114453
no_of_ratings   0.060954       1.000000        0.008429      0.003657
discount_price  0.114363       0.008429        1.000000      0.964601
actual_price    0.114453       0.003657        0.964601      1.000000


=== Data Completeness ===
Percentage of complete rows: 94.38%


=== Top 3 Products with the Highest Rating ===
Top 3 Products with the Highest Rating:
                

C:\Users\N I T R O   5\AppData\Local\Temp\ipykernel_37400\801549112.py:39: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()


Transformed data saved for All Appliances.csv.

=== Analysis for All Books.csv ===
=== Basic Statistics ===
Basic Statistics:
       ratings  no_of_ratings  discount_price  actual_price
count      0.0            0.0             0.0           0.0
mean       NaN            NaN             NaN           NaN
std        NaN            NaN             NaN           NaN
min        NaN            NaN             NaN           NaN
25%        NaN            NaN             NaN           NaN
50%        NaN            NaN             NaN           NaN
75%        NaN            NaN             NaN           NaN
max        NaN            NaN             NaN           NaN


=== Missing Values Analysis ===
Missing Values Analysis:
                Missing Values  Percentage
name                       0.0         NaN
main_category              0.0         NaN
sub_category               0.0         NaN
image                      0.0         NaN
link                       0.0         NaN
ratings          

...

C:\Users\N I T R O   5\AppData\Local\Temp\ipykernel_37400\801549112.py:39: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()


Transformed data saved for Watches.csv.

=== Analysis for Western Wear.csv ===
=== Basic Statistics ===
Basic Statistics:
            ratings  no_of_ratings  discount_price  actual_price
count  15646.000000   19200.000000    19200.000000  19200.000000
mean       3.774102     115.528542      605.543257   1616.823514
std        0.722897     490.934025      464.262938   1031.958070
min        1.000000       0.000000        0.000000      0.000000
25%        3.400000       1.000000      349.000000    999.000000
50%        3.800000       8.000000      499.000000   1400.000000
75%        4.100000      48.000000      750.000000   1999.000000
max        5.000000   25809.000000    14098.000000  49999.000000


=== Missing Values Analysis ===
Missing Values Analysis:
                Missing Values  Percentage
name                         0    0.000000
main_category                0    0.000000
sub_category                 0    0.000000
image                        0    0.000000
link               

C:\Users\N I T R O   5\AppData\Local\Temp\ipykernel_37400\801549112.py:39: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()


=== Analysis for Womens Fashion.csv ===
=== Basic Statistics ===
Basic Statistics:
           ratings  no_of_ratings  discount_price  actual_price
count  1873.000000    2112.000000     2112.000000   2112.000000
mean      3.909130     429.457860      811.657401   2187.363480
std       0.527195    2111.540106     2229.037946   3063.651103
min       1.000000       0.000000        0.000000      0.000000
25%       3.600000       7.000000      289.000000    999.000000
50%       3.900000      51.000000      469.000000   1499.000000
75%       4.200000     288.000000      799.000000   2499.000000
max       5.000000   67255.000000    64111.000000  70522.100000


=== Missing Values Analysis ===
Missing Values Analysis:
                Missing Values  Percentage
name                         0    0.000000
main_category                0    0.000000
sub_category                 0    0.000000
image                        0    0.000000
link                         0    0.000000
ratings                 

C:\Users\N I T R O   5\AppData\Local\Temp\ipykernel_37400\801549112.py:39: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()
C:\Users\N I T R O   5\AppData\Local\Temp\ipykernel_37400\801549112.py:39: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()
